In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv
              
import requests
              
import json
from pandas.io.json import json_normalize

import numpy as np



In [2]:
load_dotenv()
client_id = os.getenv('foursquare_id')
client_secret = os.getenv('foursquare_secret')
version = '20200508'


In [3]:
dallas_df = pd.read_csv('./Data/dfw_demographics.csv')

In [4]:
dallas_df.head()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,75001,Addison,32.96,-96.84,12414,57633.0,258200.0,0.092103,0.366306,0.038184,0.332532,0.144217,0.023251,0.003407
1,75002,West Allen,33.08,-96.61,63140,91632.0,186100.0,0.052425,0.416282,0.085043,0.310966,0.109292,0.015340,0.010652
2,75006,Central Carrolton,32.95,-96.89,46364,56985.0,146800.0,0.252946,0.383821,0.066069,0.209083,0.075295,0.010197,0.002590
3,75007,North Carrollton,33.00,-96.90,51624,78934.0,169400.0,0.101158,0.426112,0.081565,0.288381,0.075876,0.014833,0.012075
4,75009,Celina,33.30,-96.80,8785,87581.0,190500.0,0.073749,0.525198,0.106760,0.215979,0.069008,0.006146,0.003161


In [5]:
dallas_df.tail()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
265,76487,Poolville,33.000,-97.90,2507,54872.0,132800.0,0.151007,0.605369,0.070470,0.141611,0.031544,0.000000,0.0
266,76490,Whitt,32.957,-98.02,91,107344.0,32500.0,0.172840,0.753086,0.074074,0.000000,0.000000,0.000000,0.0
267,76623,Avalon,32.300,-96.80,242,41250.0,80700.0,0.410526,0.536842,0.000000,0.052632,0.000000,0.000000,0.0
268,76651,Italy,32.200,-96.90,3300,50457.0,90300.0,0.224719,0.588764,0.048989,0.106067,0.029663,0.001798,0.0
269,76670,Milford,32.100,-97.00,1377,49667.0,86700.0,0.192175,0.660529,0.036824,0.100115,0.005754,0.004603,0.0


In [6]:
def get_nearby_venues(names, latitudes, longitudes, radius= 16100, section = 'shops', limit = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius,
            section,
            limit)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        except:
            print(f'{name} has no restuarants')

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)  

In [7]:
dallas_venues = get_nearby_venues(names = dallas_df['Neighborhood'],
                                   latitudes = dallas_df['lat'],
                                   longitudes = dallas_df['lng']
                                  )

Addison
West Allen
Central Carrolton
North Carrollton
Celina
Far North Carrollton
East Allen
Coppell
West Flower Mound
(Quail Creek, Cross Bend) - Plano
(Deerfield West, Normandy Estates) - Plano
(Ridgeview, Russel Creek Park) - Plano
East Flower Mound
South Rockwall
West Frisco
West Frisco has no restuarants
East Frisco
East Frisco has no restuarants
(Cottonwood Valley, Arts District) - Irving
Las Colinas - Irving
Northeast Garland
(Almeta, Bonite, Bella Vista) - Garland
(Forest Crest, Park Groves, Creekside Village) - Garland
Bass Park Area - Garland
(Firewheel Hills, Picket Place, Camelot) - Garland
Sachse
North Grand Prairie
South Grand Prairie
Southwest Grand Prairie
Estes Park Area - Grand Prairie
The Colony
Northeast to Southeast Lewisville
(Lamar Brown, Irving Lake, Trinity Oaks) - Irving
(Garden Oaks, Plymoth Park) - Irving
(Hillcrest Oaks, Arts District, Woodhaven) - Irving
(Hackberry Creek, Valley Ranch) - Irving
Lake Dallas
West to Southwest Lewisville
Little Elm
Southeast 

In [8]:
dallas_venues.shape

(25060, 7)

In [9]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.96,-96.84,Whole Foods Market,32.953183,-96.821189,Grocery Store
1,Addison,32.96,-96.84,Trader Joe's,32.950679,-96.801981,Grocery Store
2,Addison,32.96,-96.84,Spec's,32.953851,-96.830795,Liquor Store
3,Addison,32.96,-96.84,Galleria Dallas,32.930022,-96.819643,Shopping Mall
4,Addison,32.96,-96.84,QuikTrip,32.933408,-96.802600,Convenience Store


In [10]:
dallas_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
25055,Milford,32.1,-97.0,T. B. Bond Pharmacy,32.010567,-97.130383,Pharmacy
25056,Milford,32.1,-97.0,steel stylin,32.009147,-97.130423,Health & Beauty Service
25057,Milford,32.1,-97.0,Redbox,32.007415,-97.130185,Video Store
25058,Milford,32.1,-97.0,7-Eleven,32.008289,-97.093517,Convenience Store
25059,Milford,32.1,-97.0,Outlets at Hillsboro,32.013207,-97.094421,Shopping Mall


In [11]:
dallas_venues.to_csv('./Data/dallas_venues.csv', index = False)

In [12]:
houston_df = pd.read_csv('./Data/houston_demographics.csv')

In [13]:
houston_df.head()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
0,77002,"Downtown, Midtown",29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687
1,77003,East Downtown,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715
2,77004,"South Central, Greater Third Ward",29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400
3,77005,West University Place,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904
4,77006,"Montrose, Hyde Park",29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589


In [14]:
houston_df.tail()

,Zip_Code,Neighborhood,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree
229,77665,Winnie,29.80,-94.30,6446,50149.0,116400.0,0.249186,0.575733,0.035831,0.100434,0.030130,0.005157,0.003529
230,78931,Bleiblerville,30.03,-96.44,323,67917.0,116400.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
231,78933,Cat Spring,29.80,-96.40,1433,59974.0,225700.0,0.049579,0.610851,0.037418,0.211413,0.061740,0.020580,0.008419
232,78944,Industry,30.00,-96.50,562,47083.0,154900.0,0.230461,0.561122,0.068136,0.036072,0.102204,0.002004,0.000000
233,78950,New Ulm,29.90,-96.50,2038,49261.0,170900.0,0.086690,0.559408,0.134115,0.150943,0.056604,0.012239,0.000000


In [15]:
houston_venues = get_nearby_venues(names = houston_df['Neighborhood'],
                                   latitudes = houston_df['lat'],
                                   longitudes = houston_df['lng']
                                  )

Downtown, Midtown
East Downtown
South Central, Greater Third Ward
West University Place
Montrose, Hyde Park
Rice Military, Memorial Park
Shady Acres, Greater Heights, Lazybrook
Northside Village, Woodland Heights
George R Brown
Greater East End, Second ward, Magnolia Park
Harrisburg, Manchester
North Shore
Silverglen, Sablechase
Cloverleaf
East Little York, Homestead
Meadowbrook, Allendale
Central Northwest
River Oaks, Fourth Ward
Greater Fifth Ward, Denver Harbor, Port Houston
Greater OST
Independence Heights
Greater Eastwood, Lawndale, Wayside, Gulfgate, Pine Valley
Memorial Villages, Briargrove Park
Braeswood Place, Willowbend Area
Kashmere Gardens
River Oaks, Bellaire Junction
Trinity, Houston Gardens, Settegast
Jacinto City
Texas Medical Center
Brays Oaks, Braeburn Gardens
Greenbriar Colony, Aldine Estates
Southeast Houston, Southpark
Edgebrook
Westbury, Southeast Houston
Chinatown, Sharpstown
Aldine
Fallbrook, Forestwood
High Meadows, Fairgreen
Fairbanks, Northwest Crossing
Lakes

In [16]:
houston_venues.shape

(21142, 7)

In [17]:
houston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.748694,-95.376949,Liquor Store
1,"Downtown, Midtown",29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Market
2,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.744248,-95.392225,Liquor Store
3,"Downtown, Midtown",29.75,-95.37,Whole Foods Market,29.757994,-95.397643,Grocery Store
4,"Downtown, Midtown",29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store


In [18]:
houston_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
21137,New Ulm,29.9,-96.5,Professional Processor,29.892169,-96.490388,Business Service
21138,New Ulm,29.9,-96.5,Family Dollar,29.971891,-96.500930,Discount Store
21139,New Ulm,29.9,-96.5,Nevins LLC,29.854891,-96.375605,Furniture / Home Store
21140,New Ulm,29.9,-96.5,Zebras For Sale,29.957574,-96.617578,Pet Store
21141,New Ulm,29.9,-96.5,Gagne's Landscaping,29.875202,-96.359582,Construction & Landscaping


In [19]:
houston_venues.to_csv('./Data/houston_venues.csv', index = False)

In [20]:
dallas_venues = pd.read_csv('./Data/dallas_venues.csv')

In [21]:
dallas_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addison,32.96,-96.84,Whole Foods Market,32.953183,-96.821189,Grocery Store
1,Addison,32.96,-96.84,Trader Joe's,32.950679,-96.801981,Grocery Store
2,Addison,32.96,-96.84,Spec's,32.953851,-96.830795,Liquor Store
3,Addison,32.96,-96.84,Galleria Dallas,32.930022,-96.819643,Shopping Mall
4,Addison,32.96,-96.84,QuikTrip,32.933408,-96.802600,Convenience Store


In [22]:
ten_or_less_unique_venues = dallas_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').nunique() < 10

In [23]:
ten_or_less_unique_venues = ten_or_less_unique_venues[ten_or_less_unique_venues['Venue Category'] == True]

In [24]:
ten_or_less_unique_venues

,Neighborhood,Venue Category
Neighborhood,,
Lipan,True,True
Poolville,True,True
Rosser,True,True
Whitt,True,True


In [25]:
neighborhoods_to_drop = list(ten_or_less_unique_venues.index.values)

In [26]:
dallas_venues = dallas_venues[~dallas_venues['Neighborhood'].isin(neighborhoods_to_drop)]

In [27]:
dallas_df = dallas_df[~dallas_df['Neighborhood'].isin(neighborhoods_to_drop)] 

In [28]:
print('There are {} uniques categories.'.format(len(dallas_venues['Venue Category'].unique())))

There are 115 uniques categories.


In [29]:
dallas_onehot = pd.get_dummies(dallas_venues[['Venue Category']], prefix="", prefix_sep="")

dallas_onehot['Neighborhood'] = dallas_venues['Neighborhood'] 

fixed_columns = [dallas_onehot.columns[-1]] + list(dallas_onehot.columns[:-1])
dallas_onehot = dallas_onehot[fixed_columns]

dallas_onehot.head()


,Neighborhood,ATM,Accessories Store,Adult Boutique,Antique Shop,Arts & Crafts Store,Astrologer,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,Board Shop,Bookstore,Boutique,Bridal Shop,Business Service,Camera Store,Candy Store,Carpet Store,Check Cashing Service,Cheese Shop,Child Care Service,Clothing Store,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Department Store,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,Fireworks Store,Fishing Store,Flea Market,Floating Market,Flower Shop,Food & Drink Shop,Food Service,Frame Store,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gift Shop,Gourmet Shop,Grocery Store,Gun Shop,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hobby Shop,Home Service,Hunting Supply,IT Services,Insurance Office,Jewelry Store,Kids Store,Kitchen Supply Store,Laundromat,Lawyer,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Marijuana Dispensary,Market,Massage Studio,Mattress Store,Medical Supply Store,Men's Store,Mobile Phone Shop,Mobility Store,Motorcycle Shop,Motorsports Shop,Music Store,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Pawn Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pop-Up Shop,Print Shop,Public Bathroom,Rental Service,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Vape Store,Video Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store
0,Addison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Addison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Addison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Addison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Addison,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
dallas_onehot.shape

(24983, 116)

In [31]:
dallas_grouped = dallas_onehot.groupby('Neighborhood').mean().reset_index()
dallas_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Antique Shop,Arts & Crafts Store,Astrologer,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,Board Shop,Bookstore,Boutique,Bridal Shop,Business Service,Camera Store,Candy Store,Carpet Store,Check Cashing Service,Cheese Shop,Child Care Service,Clothing Store,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Department Store,Discount Store,Drugstore,Dry Cleaner,Electronics Store,Event Service,Fabric Shop,Farmers Market,Fireworks Store,Fishing Store,Flea Market,Floating Market,Flower Shop,Food & Drink Shop,Food Service,Frame Store,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gift Shop,Gourmet Shop,Grocery Store,Gun Shop,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hobby Shop,Home Service,Hunting Supply,IT Services,Insurance Office,Jewelry Store,Kids Store,Kitchen Supply Store,Laundromat,Lawyer,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Marijuana Dispensary,Market,Massage Studio,Mattress Store,Medical Supply Store,Men's Store,Mobile Phone Shop,Mobility Store,Motorcycle Shop,Motorsports Shop,Music Store,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Pawn Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Pop-Up Shop,Print Shop,Public Bathroom,Rental Service,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Vape Store,Video Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store
0,"(Almeta, Bonite, Bella Vista) - Garland",0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.09,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.09,0.0,0.0,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.01,0.31,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.03,0.09,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.04,0.02,0.01,0.00,0.01,0.14,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.02,0.0
1,"(Arlington Heights, Monticello, West Beyer) - ...",0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.23,0.0,0.0,0.02,0.04,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.03,0.02,0.01,0.01,0.17,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.03,0.13,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.01,0.02,0.00,0.01,0.00,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.03,0.01,0.0
2,"(Bluffview, Greenway Parks) - Dallas",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.13,0.0,0.0,0.01,0.00,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.03,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.02,0.09,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.03,0.02,0.01,0.00,0.00,0.14,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.03,0.0
3,"(Casa Linda, Lockwood) - Dallas",0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.07,0.0,0.0,0.01,0.00,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.02,0.00,0.03,0.28,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.03,0.09,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.05,0.02,0.01,0.00,0.02,0.12,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.02,0.0
4,"(Casa View, Eastwood Hills) - Dallas",0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.01,

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

dallas_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
dallas_neighborhoods_venues_sorted['Neighborhood'] = dallas_grouped['Neighborhood']

for ind in np.arange(dallas_grouped.shape[0]):
    dallas_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dallas_grouped.iloc[ind, :], num_top_venues)

dallas_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"(Almeta, Bonite, Bella Vista) - Garland",Grocery Store,Supermarket,Convenience Store,Pharmacy,Big Box Store,Warehouse Store,Shopping Mall,Pet Store,Electronics Store,Wine Shop
1,"(Arlington Heights, Monticello, West Beyer) - ...",Convenience Store,Grocery Store,Pharmacy,Big Box Store,Discount Store,Supermarket,Liquor Store,Warehouse Store,Furniture / Home Store,Shipping Store
2,"(Bluffview, Greenway Parks) - Dallas",Grocery Store,Supermarket,Convenience Store,Pharmacy,Big Box Store,Warehouse Store,Liquor Store,Gourmet Shop,Wine Shop,Shopping Mall
3,"(Casa Linda, Lockwood) - Dallas",Grocery Store,Supermarket,Pharmacy,Convenience Store,Warehouse Store,Big Box Store,Shopping Mall,Gourmet Shop,Pet Store,Shopping Plaza
4,"(Casa View, Eastwood Hills) - Dallas",Grocery Store,Supermarket,Pharmacy,Convenience Store,Big Box Store,Warehouse Store,Shopping Mall,Gourmet Shop,Pet Store,Shopping Plaza


In [34]:
dallas_combined_df = pd.concat([dallas_df.set_index('Neighborhood'),dallas_neighborhoods_venues_sorted.set_index('Neighborhood')], axis = 1, sort = False)

In [35]:
dallas_combined_df.reset_index().to_csv('./Data/dallas_combined_df_with_venue_labels.csv', index = False)

In [36]:
dallas_combine_no_labels = pd.concat([dallas_df.set_index('Neighborhood'), dallas_grouped.set_index('Neighborhood')], axis = 1, sort = False)

In [37]:
dallas_combine_no_labels.reset_index().to_csv('./Data/dallas_combined_df.csv', index = False)

In [38]:
houston_venues = pd.read_csv('./Data/houston_venues.csv')

In [39]:
houston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.748694,-95.376949,Liquor Store
1,"Downtown, Midtown",29.75,-95.37,Phoenicia Specialty Foods,29.754502,-95.361760,Market
2,"Downtown, Midtown",29.75,-95.37,"Spec's Wines, Spirits & Finer Foods",29.744248,-95.392225,Liquor Store
3,"Downtown, Midtown",29.75,-95.37,Whole Foods Market,29.757994,-95.397643,Grocery Store
4,"Downtown, Midtown",29.75,-95.37,H-E-B,29.737913,-95.402618,Grocery Store


In [40]:
ten_or_less_unique_venues = houston_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').nunique() < 10

In [41]:
ten_or_less_unique_venues = ten_or_less_unique_venues[ten_or_less_unique_venues['Venue Category'] == True]

In [42]:
ten_or_less_unique_venues

,Neighborhood,Venue Category
Neighborhood,,
Cat Spring,True,True
Central Bolivar Peninsula,True,True
Damon,True,True
Devers,True,True
East Bolivar Peninsula,True,True
East Cleveland,True,True
Industry,True,True
New Ulm,True,True
Romayor,True,True


In [43]:
neighborhoods_to_drop = list(ten_or_less_unique_venues.index.values)

In [44]:
houston_venues = houston_venues[~houston_venues['Neighborhood'].isin(neighborhoods_to_drop)]

In [45]:
houston_df = houston_df[~houston_df['Neighborhood'].isin(neighborhoods_to_drop)] 

In [46]:
print('There are {} uniques categories.'.format(len(houston_venues['Venue Category'].unique())))

There are 116 uniques categories.


In [47]:
houston_onehot = pd.get_dummies(houston_venues[['Venue Category']], prefix="", prefix_sep="")

houston_onehot['Neighborhood'] = houston_venues['Neighborhood'] 

fixed_columns = [houston_onehot.columns[-1]] + list(houston_onehot.columns[:-1])
houston_onehot = houston_onehot[fixed_columns]

houston_onehot.head()

,Neighborhood,ATM,Accessories Store,Antique Shop,Arts & Crafts Store,Astrologer,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,Board Shop,Bookstore,Boutique,Bridal Shop,Business Service,Camera Store,Candy Store,Carpet Store,Check Cashing Service,Cheese Shop,Child Care Service,Chocolate Shop,Clothing Store,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Department Store,Discount Store,Dry Cleaner,Electronics Store,Entertainment Service,Event Service,Fabric Shop,Farmers Market,Fireworks Store,Fish Market,Fishing Store,Flea Market,Floating Market,Flower Shop,Food & Drink Shop,Food Service,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gift Shop,Gourmet Shop,Grocery Store,Gun Shop,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hobby Shop,Home Service,Hunting Supply,IT Services,Insurance Office,Jewelry Store,Kids Store,Kitchen Supply Store,Laundromat,Lawyer,Leather Goods Store,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Marijuana Dispensary,Market,Massage Studio,Mattress Store,Medical Supply Store,Men's Store,Mobile Phone Shop,Motorcycle Shop,Motorsports Shop,Music Store,Notary,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Pawn Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Print Shop,Public Bathroom,Record Shop,Recording Studio,Rental Service,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Stationery Store,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Vape Store,Video Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store
0,"Downtown, Midtown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Downtown, Midtown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Downtown, Midtown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Downtown, Midtown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Downtown, Midtown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
houston_onehot.shape

(21046, 117)

In [49]:
houston_grouped = houston_onehot.groupby('Neighborhood').mean().reset_index()
houston_grouped.head()

,Neighborhood,ATM,Accessories Store,Antique Shop,Arts & Crafts Store,Astrologer,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,Board Shop,Bookstore,Boutique,Bridal Shop,Business Service,Camera Store,Candy Store,Carpet Store,Check Cashing Service,Cheese Shop,Child Care Service,Chocolate Shop,Clothing Store,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Department Store,Discount Store,Dry Cleaner,Electronics Store,Entertainment Service,Event Service,Fabric Shop,Farmers Market,Fireworks Store,Fish Market,Fishing Store,Flea Market,Floating Market,Flower Shop,Food & Drink Shop,Food Service,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gift Shop,Gourmet Shop,Grocery Store,Gun Shop,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hobby Shop,Home Service,Hunting Supply,IT Services,Insurance Office,Jewelry Store,Kids Store,Kitchen Supply Store,Laundromat,Lawyer,Leather Goods Store,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Marijuana Dispensary,Market,Massage Studio,Mattress Store,Medical Supply Store,Men's Store,Mobile Phone Shop,Motorcycle Shop,Motorsports Shop,Music Store,Notary,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Pawn Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Print Shop,Public Bathroom,Record Shop,Recording Studio,Rental Service,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Stationery Store,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Vape Store,Video Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store
0,North Humble,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.00,0.03,0.0,0.01,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.01,0.05,0.02,0.0,0.03,0.14,0.0,0.03,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.02,0.00,0.03,0.01,0.0,0.00,0.07,0.01,0.00,0.0,0.0,0.01,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.01,0.0,0.0,0.00,0.00,0.02,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.01,0.00,0.0,0.06,0.16,0.0,0.00,0.0,0.0,0.0,0.0,0.06,0.01,0.0,0.00,0.0,0.00,0.02,0.04,0.0,0.05,0.01,0.0,0.00,0.0,0.0,0.01,0.00,0.02,0.01,0.0
1,"Addicks, Park Ten",0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00,0.06,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.05,0.00,0.0,0.02,0.03,0.0,0.01,0.0,0.00,0.03,0.0,0.0,0.0,0.0,0.00,0.0,0.01,0.01,0.03,0.00,0.00,0.00,0.0,0.01,0.24,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.07,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.02,0.09,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.0,0.00,0.01,0.05,0.0,0.13,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.05,0.02,0.0
2,Aldine,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.01,0.09,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.03,0.00,0.0,0.03,0.06,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.00,0.00,0.00,0.01,0.00,0.0,0.00,0.12,0.00,0.01,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.02,0.00,0.0,0.01,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.03,0.00,0.0,0.04,0.22,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.01,0.0,0.01,0.02,0.01,0.0,0.11,0.00,0.0,0.00,0.0,0.0,0.00,0.01,0.02,0.00,0.0
3,Alief,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00,0.08,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.01,0.00,0.01,0.00,0.02,0.00,0.0,0.02,0.00,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.01,0.01,0.01,0.01,0.00,0.0,0.02,0.25,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.06,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.05,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.09,0.0,0.01,0.01,0.03,0.0,0.17,0.01,0.0,0.00,0.0,0.0,0.00,0.00,0.06,0.02,0.0
4,Alvin,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.00,0.01,0.0,0.00,0.0,0.0,0.06,0.0,0.0,0.01,0.01,0.0,0.0,0.00,0.03,0.00,0.09,0.05,0.02,0.0,0

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

houston_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
houston_neighborhoods_venues_sorted['Neighborhood'] = houston_grouped['Neighborhood']

for ind in np.arange(houston_grouped.shape[0]):
    houston_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_grouped.iloc[ind, :], num_top_venues)

houston_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Humble,Pharmacy,Discount Store,Grocery Store,Shipping Store,Pet Store,Supermarket,Convenience Store,Sporting Goods Shop,Big Box Store,Electronics Store
1,"Addicks, Park Ten",Grocery Store,Supermarket,Pharmacy,Liquor Store,Big Box Store,Shopping Mall,Warehouse Store,Convenience Store,Sporting Goods Shop,Discount Store
2,Aldine,Pharmacy,Grocery Store,Supermarket,Big Box Store,Discount Store,Shipping Store,Pet Store,Convenience Store,Department Store,Paper / Office Supplies Store
3,Alief,Grocery Store,Supermarket,Shopping Mall,Big Box Store,Warehouse Store,Liquor Store,Pharmacy,Sporting Goods Shop,Department Store,Wine Shop
4,Alvin,Home Service,Construction & Landscaping,Mobile Phone Shop,Grocery Store,Discount Store,Business Service,Video Store,Convenience Store,Furniture / Home Store,Insurance Office


In [51]:
houston_combined_df = pd.concat([houston_df.set_index('Neighborhood'), houston_neighborhoods_venues_sorted.set_index('Neighborhood')], axis = 1, sort = False)

In [52]:
houston_combined_df.head()

,Zip_Code,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
"Downtown, Midtown",77002,29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687,Grocery Store,Pharmacy,Supermarket,Liquor Store,Pet Store,Big Box Store,Shopping Mall,Warehouse Store,Arts & Crafts Store,Shipping Store
East Downtown,77003,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715,Grocery Store,Pharmacy,Supermarket,Pet Store,Liquor Store,Shopping Mall,Sporting Goods Shop,Big Box Store,Department Store,Shipping Store
"South Central, Greater Third Ward",77004,29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400,Grocery Store,Pharmacy,Supermarket,Liquor Store,Big Box Store,Pet Store,Shopping Mall,Department Store,Discount Store,Shipping Store
West University Place,77005,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904,Grocery Store,Supermarket,Pharmacy,Shopping Mall,Big Box Store,Liquor Store,Pet Store,Warehouse Store,Department Store,Wine Shop
"Montrose, Hyde Park",77006,29.74,-95.39,19664,67458.0,327300.0,0.033678,0.264526,0.030957,0.339017,0.172018,0.101215,0.058589,Grocery Store,Pharmacy,Supermarket,Shopping Mall,Liquor Store,Big Box Store,Pet Store,Sporting Goods Shop,Warehouse Store,Department Store


In [53]:
houston_combined_df.reset_index().to_csv('./Data/houston_combined_with_venue_labels_df.csv', index = False)

In [54]:
houston_combine_no_labels = pd.concat([houston_df.set_index('Neighborhood'), houston_grouped.set_index('Neighborhood')], axis = 1, sort = False)

In [55]:
houston_combine_no_labels.head()

,Zip_Code,lat,lng,population,median_household_income,median_home_value,Less Than High School,High School Graduate,Associate's Degree,Bachelor's Degree,Master's Degree,Professional Degree,Doctorate Degree,ATM,Accessories Store,Antique Shop,Arts & Crafts Store,Astrologer,Auto Garage,Auto Workshop,Baby Store,Beer Store,Big Box Store,Board Shop,Bookstore,Boutique,Bridal Shop,Business Service,Camera Store,Candy Store,Carpet Store,Check Cashing Service,Cheese Shop,Child Care Service,Chocolate Shop,Clothing Store,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Department Store,Discount Store,Dry Cleaner,Electronics Store,Entertainment Service,Event Service,Fabric Shop,Farmers Market,Fireworks Store,Fish Market,Fishing Store,Flea Market,Floating Market,Flower Shop,Food & Drink Shop,Food Service,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gift Shop,Gourmet Shop,Grocery Store,Gun Shop,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hobby Shop,Home Service,Hunting Supply,IT Services,Insurance Office,Jewelry Store,Kids Store,Kitchen Supply Store,Laundromat,Lawyer,Leather Goods Store,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Marijuana Dispensary,Market,Massage Studio,Mattress Store,Medical Supply Store,Men's Store,Mobile Phone Shop,Motorcycle Shop,Motorsports Shop,Music Store,Notary,Optical Shop,Other Repair Shop,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Pawn Shop,Perfume Shop,Pet Store,Pharmacy,Photography Studio,Print Shop,Public Bathroom,Record Shop,Recording Studio,Rental Service,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Smoothie Shop,Sporting Goods Shop,Stationery Store,Supermarket,Supplement Shop,Tailor Shop,Thrift / Vintage Store,Toy / Game Store,Vape Store,Video Game Store,Video Store,Warehouse Store,Wine Shop,Women's Store
"Downtown, Midtown",77002,29.75,-95.37,16793,64093.0,208600.0,0.265749,0.416079,0.049119,0.152974,0.071696,0.021696,0.022687,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.01,0.0,0.0,0.02,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.01,0.01,0.00,0.0,0.01,0.21,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.01,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.04,0.0,0.01,0.02,0.01,0.0,0.12,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.02,0.0
East Downtown,77003,29.75,-95.35,10508,36412.0,162600.0,0.318136,0.369392,0.038623,0.165674,0.061130,0.029331,0.017715,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.02,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.00,0.0,0.0,0.02,0.00,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.01,0.01,0.01,0.0,0.01,0.22,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.01,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.05,0.0,0.01,0.02,0.03,0.0,0.10,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.0
"South Central, Greater Third Ward",77004,29.72,-95.38,32692,41983.0,191700.0,0.157673,0.345377,0.040444,0.230725,0.137109,0.060271,0.028400,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.01,0.0,0.0,0.03,0.02,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.01,0.01,0.00,0.0,0.01,0.21,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.01,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.14,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.04,0.0,0.01,0.02,0.01,0.0,0.12,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.0
West University Place,77005,29.72,-95.42,25528,159732.0,754000.0,0.016033,0.122208,0.040450,0.334924,0.227403,0.184077,0.074904,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [56]:
houston_combine_no_labels.reset_index().to_csv('./Data/houston_combined_df.csv', index = False)